**(Run this cell to define useful Latex macros)**
\\[
\newcommand{\card}[1]{\left\lvert#1\right\rvert}
\newcommand{\condbar}[0]{\,\big|\,}
\newcommand{\eprob}[1]{\widehat{\text{Pr}}\left[#1\right]}
\newcommand{\fpartial}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\ffpartial}[2]{\frac{\partial^2 #1}{\partial {#2}^2}}
\newcommand{\gradient}[0]{\nabla}
\newcommand{\norm}[1]{\left\lvert\left\lvert#1\right\rvert\right\rvert}
\newcommand{\prob}[1]{\text{Pr}\left[#1\right]}
\newcommand{\pprob}[2]{\text{Pr}_{#1}\left[#2\right]}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\trans}[1]{#1^\mathsf{T}}
\\]

# Lecture 01: Logistic Regression

In the previous lecture we saw how to learn a *regression* model. Regression problems involve predicting a continuous valued variable with range $-\infty, \infty$.

Another kind of problem is called *classification*. *Binary* classification is the simplest kind of classification problem. Binary classification problems have YES or NO answers. For instance, "Is this a photo of a cat?" "Will this patient die in the next 3 months?"

Classification problems can have more than one *class*. For instance, the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) is a dataset of 70,000 images of handwritten numerals: 0 through 9. Each image belongs to one of *ten* classes: it is either a zero, or a one, or a two, et cetera.

We could *try* to cast classification as a regression. For instance, we could try to use a linear regression model, and try to train it so that if the image is a 9, the linear model outputs 9.0. That will work very poorly though.

The first problem is this. What does it mean if your linear regression model -7? Or 33? Or 3.75? Do you round to the closest class? But then every negative number maps to class 0, whereas only the range $(4.5, 5.5)$ maps to class 5. That certainly seems unfair.

Also: let's say for an example $x$ your linear model outputs 3.5, but the correct class is $y = 5$. You would like to increase the output of your linear model, so that it is closer to guessing the right answer. But by increasing the output, you are also making the model think that the answers $y = 4$ and $y = 6$ are more "likely," too.

The problem is this: you're putting the classes on a one-dimensional continuum, when in fact they are discrete. There is no way to slowly change a 0 digit into a 1, and then keep changing like that from a 1 to a 2. That's fundamentally why trying to cast this problem as regression is so wrong.


You will have to install Keras for this to work! `pip install keras`. You may wish to check using `which pip` that you are using the Anaconda installed version of the pip command.

In [1]:
from keras.datasets import imdb

TOP_N_WORDS = 1000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = TOP_N_WORDS)

print(len(x_train))
print(len(x_test))

print(x_train[0])
print(y_train[0])

Using TensorFlow backend.


25000
25000
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


What we have here is a training set of 25,000 IMDB reviews, each marked as either 0 (negative sentiment) or 1 (positive sentiment). The reviews are encoded as a sequence of integers. Each integer stands for a word. So the sequence of words in a review are translated into a sequence of numbers.

The most frequent word in the dataset is assigned the number 1: presumably this is the word "the." The second most frequent word is assigned the number 2, et cetera. We have thrown out all but the thousand most common words.

For this simple project, we will not use either the order of the words in a review, nor how frequently a word appears. This is quite a loss of information, but we will see that we can still do quite a good job!

We will convert the variable-length sequences into fixed-length vectors of length 1,000. A review will have the value 1 at position $k$ if word $k$ is present somewhere in the review. The review will have a zero at position $k$ if the word is not present anywhere in the review.

This is called a *bag of words* model.


In [2]:
import numpy as np

# Transform dataset from variable-length word sequences to a binary valued dense matrix.
new_x_train = np.zeros((len(x_train), TOP_N_WORDS + 1))
# We'll use a dummy column 0 to apply an intercept theta_0 to our model. It will always have value 1.
new_x_train[:, 0] = 1.0

for example_idx, word_sequence in enumerate(x_train):
    for word_idx in word_sequence:
        new_x_train[example_idx, word_idx] = 1

new_x_test = np.zeros((len(x_test), TOP_N_WORDS + 1))
new_x_test[:, 0] = 1.0
for example_idx, word_sequence in enumerate(x_test):
    for word_idx in word_sequence:
        new_x_test[example_idx, word_idx] = 1


For this problem we're going to learn a different weight $\theta_i \in (-\infty, \infty)$ for each of the $i$ top words. We'll use what's called a *generalized linear model:*

\\[
f(x) = g\left( \sum_{i = 0}^N \theta_i x_i \right)
\\]

This is a generalized linear model because we first compute a linear function of the $x_i$, and *then* we put it through some kind of other function $g$. If $g(x) = x$, we would just be back to a regular linear model. That would give us output values in the range $(-\infty, \infty)$, which I've said I don't want.

For this binary classification problem, I would like $f(x)$ to ideally be $\prob{Y = 1 \condbar X = x}$. That is: I want $f(x)$ to be the true probability that a review expresses a positive sentiment, given that the bag of words is $x$.

In particular, to make sense as a probability, I want $g$ to squash values in the range $(-\infty, \infty)$ into the range $(0, 1)$. The function we use to compress the infinite range into a range of probabilities is called the *[logistic function](https://en.wikipedia.org/wiki/Logistic_function)*. It is defined as:

\\[
\sigma(z) = \frac{e^z}{1 + e^z}
\\]

or equivalently as

\\[
\sigma(z) = \frac{1}{e^{-z} + 1}
\\]

The first version has a straightforward interpretation, while the second version plays better with how computers do floating point calculations. Theoretically they are equivalent.


**Optional**: Why do we use the logistic function to do the squashing? Aren't there other functions that would squash the continuum into the range $(0, 1)$?

The logistic function is related to a concept called *log odds*. Let's just think about normal *odds*. If I give you one-to-ten odds that Secretariat will win a horse race, I am saying that I think it is ten times more likely that Secretariat will win than lose. That *doesn't* mean that I think Secretariat has a 10% chance of winning. That would *one-to-nine* odds.

For simplicity I will just express one-to-ten odds as 10.0. If I thought the odds were two-to-three, I would represent that as $\frac{3}{2} = 1.5$ odds.

To convert odds $o$ to a probability, I just perform the following simple calculation:

\\[
p = \frac{o}{1 + o}
\\]

Odds can only be non-negative. Negative odds would make no sense. But what if I represented odds *in the log scale*? That is, I say the *log odds* are 3.0 if I think the odds are $e^{3.0}$. Or I could say I think the log odds are -4 if I think the odds are $e^{-4}$.

How do I convert from log odds $z$ to probabilities? Simple:

\\[
p = \frac{e^z}{1 + e^z}
\\]

As you can see, the logistic function converts log odds to probabilities.

**Not optional**: Okay. We now have our binary classification problem almost all set up. Our model is parameterized by $\theta_i$, where

\\[
f(x) = \sigma\left( \sum_{i = 0}^M \theta_i x_i \right)
\\]

We will interpret this as the probability that an email with bag of words $x$ represents a positive sentiment. The number is always between zero and one.

The next step is: how do we learn this model from our previous data? We could use the MSE loss function to optimize our choice of $\theta_i$. The target would be for $f(x^{(i)}) = 1$ if $y^{(i)} = 1$. That is, if the review is positive, the ideal model would be to believe that this review has a 100% chance of being positive.

We will instead use something called the *mean cross-entropy loss*. This is sometimes abreviated CE or XE. Here is the definition:

\\[
CE(f) =
    \frac{1}{N}
    \sum_{i = 0}^N
    y^{(i)} \left(-\log{f(x^{(i)})}\right)
    +
    (1 - y^{(i)})\left(-\log{\left(1 - f(x^{(i)})\right)}\right)
\\]

This says, if $y = 1$ the review is positive. Assign an error of $-\log f(x^{(i)})$. If $f(x^{(i)}) = 1.0$, that means that you think there is a 100% the review is positive, which is correct. $\log 1.0 = 0.0$. Which means you assign yourself no error, since you are completely correct. But as $f(x^{(i)})$ approaches zero, you are getting more and more wrong. $-\log f(x^{(i)})$ will get larger and larger, so you pay more error.

On the other hand, if $y = 0$ the email is not positive. Here the error you give yourself is $-\log{(1 - f(x^{(i)}))}$. Notice that everything is symmetric and flipped. If $f(x^{(i)}) = 0.0$, you are totally correct, and you will give yourself an error of zero.


**Not optional**: Why not use the MSE? That is why not use

\\[
MSE(f) = \frac{1}{N} \sum_{i = 0}^N \left(y^{(i)} - f(x^{(i)})\right)^2
\\]

You could do this. The MSE and the CE would agree that the very best models are the ones which exactly predict $f(x^{(i)}) = y^{(i)}$. That model would have zero error under either measure.

But with machine learning we will never find a perfect model that gets everything 100% correct. We need to try to balance errors to choose the best non-perfect model.

MSE and CE disagree on how to compare two non-perfect models. Model A might be better in MSE while model B might be better in CE.

So which one is right? Here is one argument for using the cross-entropy error. It goes like this. We have observed a training dataset $X, y$. We want to choose the model $f$ which makes our past results the *most likely* results that we could have had.

Let me give a comparison to explain. If I flip a coin 100 times and get 65 heads, why do I say that the best guess is that a heads flip has probability $0.65$? I argue this is because $p = 0.65$ *maximizes* the probability of the observed dataset:

\\[
p^{65}(1-p)^{35}.
\\]

Let's verify that:

\\[
\begin{align}
\fpartial{}{p} \left(p^{65} (1-p)^{35}\right)
&=
0
\\
65p^{64}(1-p)^{35} + -35p^{65}(1-p)^{34}
&=
0
\\
65(1-p) - 35p
&=
0
\\
65 - 100p
&=
0
\\
p
&=
\frac{65}{100}
\end{align}
\\]

This principle of choosing the model that maximizes the probability of the observed prior data is called the *maximum likelihood principle* or MLE. The *likelihood* of a model is the *probability* that it assigns the observed prior data.

Let's return to binary classification. Let's use $\pprob{\theta}{Y = 1 \condbar X = x}$ instead of $f_\theta(x)$, since we want to interpret $f$ as a probability. So we want to maximize the previously observed $y$ values (takin g the $x$ values as given):

\\[
\prod_{i = 0}^N \pprob{\theta}{Y = y^{(i)} \condbar X = x^{(i)}}
\\]

Now, for the inside of this product, I'll use a trick:

\\[
\pprob{\theta}{Y = y^{(i)} \condbar X = x^{(i)}}
=
f(x^{(i)})^{y^{(i)}} (1 - f(x^{(i)}))^{(1 - y^{(i)})}
\\]

If $y^{(i)} = 1$, then the $(1 - f(x^{(i)}))$ part is raised to the zero power, and goes away. Vice versa for $y^{(i)} = 0$.

So I want to maximize:

\\[
\prod_{i = 0}^N f(x^{(i)})^{y^{(i)}} (1 - f(x^{(i)}))^{(1 - y^{(i)})}
\\]

Instead of maximizing a product, how about I maximize the log of the product? That's no different because the log function is *monotonic*: $\log a < \log b$ iff $a < b$.

\\[
\begin{align}
\log \prod_{i = 0}^N f(x^{(i)})^{y^{(i)}} (1 - f(x^{(i)}))^{(1 - y^{(i)})}
&=
\sum_{i = 0}^N \log \big( f(x^{(i)})^{y^{(i)}} (1 - f(x^{(i)}))^{(1 - y^{(i)})} \big)
\\
&=
\sum_{i = 0}^N
    \log \big( f(x^{(i)})^{y^{(i)}} \big)
    +
    \log \big((1 - f(x^{(i)}))^{(1 - y^{(i)})} \big)
\\
&=
\sum_{i = 0}^N
    y^{(i)} \log \left( f(x^{(i)}) \right)
    +
    (1 - y^{(i)}) \log \left( 1 - f(x^{(i)}) \right)
\\
&=
-CE(f)
\end{align}
\\]

What is this saying? It says: maximizing the maximum likelihood is the same as minimizing the cross-entropy loss.

Why not just directly maximize the likelihood? The reason is that products of probabilities are really bad for floating-point math, because multiplying many numbers $< 1$ quickly yields very small likelihoods. And floating-point math on your CPU has very poor precision as the magnitude of the number becomes smaller and smaller.

Working in the log space involves *adding* many (negative) logs of probabilities. This is much more accurate for floating-point math.

If you like, you can explore the wiki page on [Shannon Entropy](https://en.wikipedia.org/wiki/shannon_entropy). But this is very optional.

Okay. We now have an error function to optimize. We'll use the same strategy as last time: we'll start with a random choice of $\theta$ and perform gradient descent.

That means we need a gradient for:

\\[
y^{(i)} \left(-\log \left( f(x^{(i)}) \right)\right)
    +
    (1 - y^{(i)}) \left(-\log \left( 1 - f(x^{(i)}) \right)\right)
\\]

where

\\[
f(x) = \sigma\left( \sum_{i = 0}^M \theta_i x_i \right)
\\]

Now, if you do some calculus you can show that $\fpartial{}{z} \sigma(z) = (1 - \sigma(z))\sigma(z)$. You may verify this if you like. It only involves chain rule and differentiation of fractions and exponentionals.

Likewise, $\fpartial{}{z} \log \sigma(z) = \sigma(z)^{-1} (1 - \sigma(z))\sigma(z) = 1 - \sigma(z)$. Again, via chain rule and differentiation of logs. It may be verified that $\fpartial{}{z} \log \left(1 - \sigma(z)\right) = -\sigma(z)$.

I will use $z^{(i)} = \sum_i \theta_i x_i$ to simplify things a little bit. Let's attack the partial derivative of the error with respect to a parameter $\theta_k$:

\\[
\begin{align}
\fpartial{}{\theta_k} \left[
    y^{(i)} \left(-\log \left( f(x^{(i)}) \right)\right)
    +
    (1 - y^{(i)}) \left(-\log \left( 1 - f(x^{(i)}) \right)\right)
\right]
&
\\
=
\fpartial{}{z^{(i)}} \left[
    y^{(i)} \left(-\log \left( \sigma(z) \right)\right)
    +
    (1 - y^{(i)}) \left(-\log \left( 1 - \sigma(z) \right)\right)
\right]
\fpartial{z^{(i)}}{\theta_k}
&
\\
=
-\left[
    y^{(i)} (1 - \sigma(z^{(i)}))
    -
    (1 - y^{(i)}) \sigma(z^{(i)})
\right]
\fpartial{z^{(i)}}{\theta_k}
&
\\
=
-\left[
    y^{(i)} - \sigma(z^{(i)})
\right]
\fpartial{z^{(i)}}{\theta_k}
&
\end{align}
\\]

And of course as $z^{(i)} = \sum_i \theta_i x_i$, we have $\fpartial{z^{(i)}}{\theta_k} = x_k$. Therefore:

\\[
\fpartial{}{\theta_k} CE 
=
\frac{1}{N}
\sum_{i = 1}^N
    -\left[
        y^{(i)} - \sigma(z^{(i)})
    \right]
    x_i
=
\frac{1}{N}
\sum_{i = 1}^N
    \left[
        \sigma(z^{(i)}) - y^{(i)}
    \right]
    x_i
\\]

Well, there's nothing left but to write up the code.

In [3]:
def sigma(z):
    return 1 / (1 + np.exp(-z))

# Returns sigma(theta0 + sum x_i theta_i)
#
# Notice how this can take either a vector x, or a data matrix of x rows.
def predict(X, thetas):
    return sigma(np.dot(X, thetas))

def accuracy(X, y, thetas):
    predictions = np.round(predict(X, thetas))
    error_rate = np.sum(np.abs(y - predictions)) / X.shape[0]
    return 1 - error_rate

def avg_cross_entropy(X, y_values, thetas):
    result = 0.0
    
    num_examples = X.shape[0]
    probabilities = predict(X, thetas)
    
    result += np.sum(y_values * (-np.log(probabilities)))
    result += np.sum((1 - y_values) * (-np.log(1 - probabilities)))

    return result / num_examples

def deriv_wrt_theta_k(X, y_values, thetas, k):
    num_examples = X.shape[0]
    probabilities = predict(X, thetas)
    result = np.sum((probabilities - y_values) * X[:, k])
    return result / num_examples

def gradient_wrt_thetas(X, y_values, thetas):
    gradient = np.zeros_like(thetas)
    for k in range(len(thetas)):
        gradient[k] = deriv_wrt_theta_k(X, y_values, thetas, k)
        
    return gradient

In [10]:
NUM_EXAMPLES = new_x_train.shape[0]
def run_batch(X, y, thetas):
    lr = (X.shape[0] / NUM_EXAMPLES) * LEARNING_RATE
    new_thetas = thetas - lr * gradient_wrt_thetas(X, y, thetas)

    return new_thetas

BATCH_SIZE = 128
def run_epoch(X, y, thetas):
    for start_idx in range(0, NUM_EXAMPLES, BATCH_SIZE):
        batch_X = X[start_idx:(start_idx + BATCH_SIZE), :]
        batch_y = y[start_idx:(start_idx + BATCH_SIZE)]
        
        thetas = run_batch(batch_X, batch_y, thetas)

    loss = avg_cross_entropy(X, y, thetas)
    acc = accuracy(X, y, thetas)
    validation_acc = accuracy(new_x_test, y_test, thetas)
    print(
        f'Epoch {epoch_idx} | CE {loss:0.2f} | Acc {acc:0.2f} | Val Acc: {validation_acc:0.2f}'
    )
    
    return thetas

In [11]:
thetas_estimate = np.zeros(new_x_train.shape[1])

initial_accuracy = accuracy(new_x_train, y_train, thetas_estimate)
print(f'Initial accuracy: {initial_accuracy}')
LEARNING_RATE = 16.0
NUM_EPOCHS = 5
for epoch_idx in range(1, NUM_EPOCHS + 1):
    thetas_estimate = run_epoch(new_x_train, y_train, thetas_estimate)


Initial accuracy: 0.5
Epoch 1 | CE 0.45 | Acc 0.83 | Val Acc: 0.82
Epoch 2 | CE 0.40 | Acc 0.85 | Val Acc: 0.84
Epoch 3 | CE 0.37 | Acc 0.85 | Val Acc: 0.84
Epoch 4 | CE 0.36 | Acc 0.86 | Val Acc: 0.85
Epoch 5 | CE 0.35 | Acc 0.86 | Val Acc: 0.85
Epoch 6 | CE 0.34 | Acc 0.86 | Val Acc: 0.85


KeyboardInterrupt: 